In [47]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms ,datasets

In [58]:
class UNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        #ENCODER
        
        self.conv1 = nn.Conv2d(1, 64, kernel_size = (3, 3), stride = 1, padding = 1) # 512x512
        self.conv2 = nn.Conv2d(64, 64, kernel_size = (3, 3), stride = 1, padding = 1) # 512x512
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size = (3, 3), stride = 1, padding = 1) # 256x256
        self.conv4 = nn.Conv2d(128, 128, kernel_size = (3, 3), stride = 1, padding = 1) # 256x256
        
        self.conv5 = nn.Conv2d(128, 256, kernel_size = (3, 3), stride = 1, padding = 1) # 128x128
        self.conv6 = nn.Conv2d(256, 256, kernel_size = (3, 3), stride = 1, padding = 1) # 128x128
        
        self.conv7 = nn.Conv2d(256, 512, kernel_size = (3, 3), stride = 1, padding = 1) # 64x64
        self.conv8 = nn.Conv2d(512, 512, kernel_size = (3, 3), stride = 1, padding = 1) # 64x64
        
        self.conv9 = nn.Conv2d(512, 1024, kernel_size = (3, 3), stride = 1, padding = 1) # 32x32
        self.conv10 = nn.Conv2d(1024, 1024, kernel_size = (3, 3), stride = 1, padding = 1) # 32x32
        
        #DECODER
        
        self.
        
    def forward(self, x):
        
        #ENCODER
        
        x1 = F.relu(self.conv1(x))
        x1 = F.relu(self.conv2(x))
        
        x2 = F.max_pool2d(x, kernel_size = (2, 2), stride = 2, padding = 0) # 256x256
        x2 = F.relu(self.conv3(x))
        x2 = F.relu(self.conv4(x))
        
        x3 = F.max_pool2d(x, kernel_size = (2, 2), stride = 2, padding = 0) # 128x128
        x3 = F.relu(self.conv5(x))
        x3 = F.relu(self.conv6(x))
        
        x4 = F.max_pool2d(x, kernel_size = (2, 2), stride = 2, padding = 0) #64x64
        x4 = F.relu(self.conv7(x))
        x4 = F.relu(self.conv8(x))
        
        x5 = F.max_pool2d(x, kernel_size = (2, 2), stride = 2, padding = 0) #32x32
        x5 = F.relu(self.conv9(x))
        x5 = F.relu(self.conv10(x))
        
        return x